In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [19]:
ref_form = r'''
    Equations:
    Solution: $\left\{\begin{array}{l}5x + 4y = 6, (1) \\ 2x + 3y = 1; (2)\end{array}\right.$
    From $(1) \times 2$, get: $10x + 8y = 12. (3)$
    From $(2) \times 5$, get: $10x + 15y = 5. (4)$
    $(3)-(4)$, get: $-7y = 7,$
    Solve to get: $y = -1$.
    Substitute $y = -1$ into $(1)$, get: $5x - 4 = 6$,
    Solve to get: $x = 2$.
'''

prompt_template = r"""
    Role:
    You are Xiaopai, a professional and inspiring math teacher who can solve elementary, middle, and high school math problems. You will explain a problem one-on-one.
    Goals:
    Explain a problem like a teacher based on the given information.

    Skills:
    Can use code to assist in solving problems.
    Capable of handling various calculations.

    Workflows:
    You will explain the problem using the following framework:
        1. Identify the knowledge points involved in the problem.
        2. Determine if the given information includes the original problem and its solution process. If it does, use the original solution steps for explanations; if not, provide solutions based on similar problems.

    First Part: Original Problem and Reference Information
        Problem: {question}
        Grade Level: Middle school, your explanation should match middle school knowledge points and expression without exceeding the scope.
    Second Part: Output in the following string format
        Knowledge Point Explanation: This problem involves some xxx related knowledge points, typically solved using xxx method. ###Board Writing One: ###Explanation One: ###Board Writing Two: ###Explanation Two: ...
    Notes:
        1. Utilize reference information for solving problems; if reference information is available, it should be a similar problem.
        2. Your solving approach and explanations should closely follow the reference information, with calculations assisted by code to reduce direct problem-solving.
        3. The Knowledge Point Explanation section should state: The knowledge point of this problem is....., and the key to solving it is......
        4. Board writing is the problem-solving steps; it must be consistent with the thought process and knowledge point explanation, written in a standard answer format, divided into units of condition + conclusion.
        5. Each formula should be on its own line without displaying long continuous formulas.
        6. Board writing should primarily contain formulas with minimal Chinese text; detailed problem conditions should be abbreviated using phrases like "according to the problem."
        7. Avoid markdown syntax like ** for titles and - for lists, and avoid using \text{{}} in LaTeX formulas unless necessary.
        8. If the problem contains multiple sub-problems, show sub-problem numbers at the start of the relevant board writing section.
        9. Explanations should correspond to the board writing, emphasizing the thought process and methods involved, and be concise and precise.
        10. The response fields should only include Knowledge Point Explanation, Board Writing N, and Explanation N, with all board writing modules completed before any explanation modules; they must match in content and number.
        11. Refer to the writing style template for board writing based on the type of problem.
            {ref_form}
        12. Other Notes:
            a: Middle school inequalities have not learned to use brackets or set notation; avoid using these expressions.
            b: For middle school geometry, if proving congruent triangles or parallel lines, state the theorem used clearly after the formula.
            c: For multiple-choice questions, label options with ABCD instead of numbered brackets.

"""

chinese2eng_prompt = r"""
    Role:
        You are a professional translator specialized in mathmatical fields. Your jobs is to translate Math problem from Chinese to English
    
    Notes:
        just output the English version of the input problem, nothing else

    Problem to be translated to english:
        {problem}
"""

eng2chinese_template = r"""
    Role:
        你是一个专业的数学领域的翻译人员，你的职责是将英语题目解析翻译成中文

    Notes:
        请将英文版本的题目解析翻译成中文，保留引文题解的格式，不要输出其他任何内容

    需要翻译的内容:
        {problem}
"""

In [20]:
llm = ChatOpenAI(model='gpt-4o', max_tokens=1000)
parser = StrOutputParser()
prompt_solve = ChatPromptTemplate.from_template(prompt_template)
prompt_translate = ChatPromptTemplate.from_template(chinese2eng_prompt)
prompt_reverse = ChatPromptTemplate.from_template(eng2chinese_template)
chain_solve = prompt_solve | llm | parser
chain_translate = prompt_translate | llm | parser
chain_reverse = prompt_reverse | llm | parser

In [5]:
import json
with open("test_problem_set.json", "r") as file:
    problem_set = json.load(file)
problems = []
answers = []
for ps in problem_set:
    problems.append(ps['problem'])
    answers.append(ps['answer'])

In [31]:
english_problems = []
for problem in problems:
    output = chain_translate.invoke({"problem": problem})
    print(output)
    english_problems.append(output)

Fill in the following numbers into the brackets representing their respective sets:

-2.4, π, 2.022, -10/3, -0.15, 0, -√9, -10, -1.101 001 000 1⋯.

(1) Set of integers: { … };

(2) Set of positive real numbers: { … };

(3) Set of irrational numbers: { … }.
Given the inequality in terms of \( x \): \( a(x-1) > x + 1 - 2a \), the solution set is \( x < -1 \). Find the range of values for \( a \).
7. Define a new operation: For any real numbers \(a\) and \(b\) (where \(a \neq 0\)), \(a \otimes b = \frac{1}{a} - \frac{a - b}{a}\), the right side of the equation involves the usual addition, subtraction, and division operations. For example, \(2 \otimes 1 = \frac{1}{2} - \frac{2 - 1}{2} = 0\).

(1) The value of \(3 \otimes 4\) is ______;
(2) If the value of \(2 \otimes x\) is less than 1, find the non-negative integer solutions for \(x\).
There are 35 workers at location A and 19 workers at location B. Now, 21 more workers are to be reassigned to support, such that the number of workers at l

In [32]:
model_ansewr_english = []
for problem in english_problems:
    # print(problem)
    response = chain_solve.invoke({"question": problem, "ref_form": ref_form})
    print(response)
    model_ansewr_english.append(response)

### Knowledge Point Explanation:

The knowledge point of this problem is about classifying numbers into various sets such as integers, positive real numbers, and irrational numbers. The key to solving it is understanding the definitions and properties of these number sets and correctly identifying which numbers belong to each set.

### Board Writing One:

#### (1) Set of integers:
According to the problem, the numbers are:
\[ -2.4, \pi, 2.022, -\frac{10}{3}, -0.15, 0, -\sqrt{9}, -10, -1.1010010001\ldots \]
Among these, only whole numbers (both positive and negative, including zero) belong to the set of integers.
Condition: Checking each number:
- \(-2.4\): Not an integer (decimal)
- \(\pi\): Not an integer (irrational)
- \(2.022\): Not an integer (decimal)
- \(-\frac{10}{3}\): Not an integer (fraction)
- \(-0.15\): Not an integer (decimal)
- \(0\): Integer
- \(-\sqrt{9} = -3\): Integer
- \(-10\): Integer
- \(-1.1010010001\ldots\): Not an integer (irrational)

Conclusion: The set of int

In [35]:
model_answer_chinese = []
for eng_p in model_ansewr_english:
    response = chain_reverse.invoke({"problem": eng_p})
    print(response)
    model_answer_chinese.append(response)

### 知识点解释：

这个问题的知识点是关于将数字分类到各种集合中，如整数、正实数和无理数。解决此问题的关键是理解这些数字集合的定义和性质，并正确地识别哪些数字属于每个集合。

### 板书一：

#### (1) 整数集合：
根据问题，数字是：
\[ -2.4, \pi, 2.022, -\frac{10}{3}, -0.15, 0, -\sqrt{9}, -10, -1.1010010001\ldots \]
在这些数字中，只有整数（包括正整数、负整数和零）属于整数集合。
条件：检查每个数字：
- \(-2.4\)：不是整数（小数）
- \(\pi\)：不是整数（无理数）
- \(2.022\)：不是整数（小数）
- \(-\frac{10}{3}\)：不是整数（分数）
- \(-0.15\)：不是整数（小数）
- \(0\)：整数
- \(-\sqrt{9} = -3\)：整数
- \(-10\)：整数
- \(-1.1010010001\ldots\)：不是整数（无理数）

结论：整数集合是：
\[ \{ 0, -3, -10 \} \]

### 板书二：

#### (2) 正实数集合：
根据问题，数字是：
\[ -2.4, \pi, 2.022, -\frac{10}{3}, -0.15, 0, -\sqrt{9}, -10, -1.1010010001\ldots \]
正实数是大于零的任何数字。
条件：检查每个数字：
- \(-2.4\)：负数
- \(\pi\)：正数
- \(2.022\)：正数
- \(-\frac{10}{3}\)：负数
- \(-0.15\)：负数
- \(0\)：不是正数
- \(-\sqrt{9}\)：负数
- \(-10\)：负数
- \(-1.1010010001\ldots\)：负数

结论：正实数集合是：
\[ \{ \pi, 2.022 \} \]

### 板书三：

#### (3) 无理数集合：
根据问题，数字是：
\[ -2.4, \pi, 2.022, -\frac{10}{3}, -0.15, 0, -\sqrt{9}, -10, -1.1010010001\ldots \]
无理数是不能表示为分数且具有无限不循环小数的数字。
条件：检查每个数字：
- \(-2.4\)：有理数（有限小数）
- 

In [ ]:
from langchain_core.output_parsers import StrOutputParser
async def check(note1: str, note2: str) -> bool:
    llm = ChatOpenAI(model='gpt-4o')
    sys_prompt =    r"""
                        你是一个题解检查者，你负责检查两个题解的最终答案是否一致。注意：题解不需要一模一样，
                        只要保证最后每一问的答案都完全一致即可。
                        如果答案一致：请输出是。如果答案不一致：请输出否。请确保输出一个字，不要输出“是“与”否”意外的内容

                        题解1:
                        {note1}
                        
                        题解2:
                        {note2}
                    """
    prompt = ChatPromptTemplate.from_template(sys_prompt)
    parser = StrOutputParser()
    chain = prompt | llm | parser

    response = chain.invoke(input={"note1": note1, "note2": note2})
    return response

In [ ]:
scores = [0 for _ in range(len(model_answer_chinese))]
for i in range(len(model_answer_chinese)):
    bsl = answers[i]
    tst = model_answer_chinese[i]
    chk = await check(bsl, tst)
    if chk == '是':
        scores[i] = 1

In [ ]:
sum(scores)

In [ ]:
write2 = []
for i in range(len(problems)):
    cur = {}
    cur['problem'] = problems[i]
    cur['answer'] = model_answer_chinese[i]
    write2.append(cur)

In [ ]:
with open("outputs/model_3_output.json", 'w', encoding='utf-8') as file:
    json.dump(write2, file, ensure_ascii=False, indent=4)